In [1]:
import pandas as pd  
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

In [31]:
original_data = pd.read_csv('./eyeShadow.csv')
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
original_data['type'] = original_data['type'].map(type)

In [48]:
id_purify_data = original_data.copy()
id_purify_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,0
1,1,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,0
2,2,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,0


In [49]:
member_per_type = {0: 864, 1: 849, 2: 775, 3: 855, 4: 835}
for i in range(5):
    np.random.seed(42)
    id_purify_data.loc[id_purify_data["type"] == i, "userId"] = [np.random.randint(i*200, 200*(i+1))
                                                             for j in range(0, member_per_type[i])]


In [50]:
id_purify_data.head()

,userId,name,popId,rate,review,type
0,53,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,0
1,190,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,0
2,86,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,0
3,173,매트 아이 컬러,0,4.57,좋은 거 하나 사서 힛팬 하는 걸 목표로 프레스코 구매함. 프레스코 저렴이 많이 가...,0
4,111,매트 아이 컬러,0,4.57,한창 섀도우에 빠져 있을때 뭐 살지 고민하다가 평이 좋아서 진저 구매해서 사용해 봤...,0


In [51]:
id_purify_data = id_purify_data.sort_values('userId')

In [52]:
id_purify_data.head(3)

,userId,name,popId,rate,review,type
315,0,네이키드 히트,7,4.40,외국 브랜드 섀도우라서 로드샵의 발색을 생각했다면 좀 당황하실 수도 있어요. 색은 ...,0
3019,0,롱웨어 크림 섀도우 스틱,68,4.31,발림성 굳굳 오래 놔둬도 잘 안굳어서 좋아요\r\n애교랑 언더쪽에 음영감 넣고싶을때...,0
1911,0,플레이 컬러 아이즈 체리 블라썸,43,4.30,컬러 완전취저\r\n4계절 계속 쓰기 좋은 컬러로 구성되 있어서 짱이에요ㅎㅎ\r\n...,0


In [110]:
def reviewData(type=0):
    review_data = ['' for i in range(96)]
    review_type_popid_data = pd.DataFrame(id_purify_data[["popId", "type", "review"]])
    review_type_popid_data = review_type_popid_data.reset_index(drop=True)
    review_type_popid_data = review_type_popid_data.loc[review_type_popid_data["type"]==type,:]
    
    for index, row in review_type_popid_data.iterrows():
        review_data[row["popId"]] += row["review"]
   
    return review_data

In [111]:
review_data = reviewData(0)

In [112]:
len(review_data)

96

In [125]:
id_to_name = original_data[["popId", "name"]].drop_duplicates()
id_to_name = id_to_name.reset_index(drop=True)
id_to_name = id_to_name.drop(columns="popId", axis=1)
id_to_name

,name
0,매트 아이 컬러
1,아이섀도우
2,피스 매칭 섀도우 매트
3,모던 섀도우 이탈프리즘 - 쉬머
4,새틴 아이컬러
5,피스매칭 12구 섀도우 팔레트
6,문더스트 아이섀도우 파 아 포피에르
7,네이키드 히트
8,매그니피센트 메탈 글리터 앤 글로우 리퀴드 아이섀도우
9,스위트 피치 아이 섀도우 컬렉션


In [127]:
name_to_id = original_data[["popId", "name"]].drop_duplicates()
name_to_id = name_to_id.set_index("name", drop=True )